In [1]:
import numpy
from matplotlib import pyplot, cm, rcParams
from mpl_toolkits.mplot3d import Axes3D
#%matplotlib inline
rcParams['font.size'] = 14
from matplotlib import animation
from IPython.display import HTML

In [20]:
# Space Discretizations
Lx = 10                      # length in x direction (horizontal)
Ly = 4                      # length in y direction (vertical)
nx = 251                     # number of discretizations in x
ny = 151                     # number of discretizations in y
dx = Lx/(nx-1)               # spatial increment in x
dy = Ly/(ny-1)               # spatial increment in y
x = numpy.linspace(0,Lx,nx) # array for plotting in x
y = numpy.linspace(0,Ly,ny) # array for plotting in y
X,Y = numpy.meshgrid(x,y)   # mesh grid for plotting

#sigma = 0.2
c = 1 # approximate speed of sound [m/s]
# from http://www.engineeringtoolbox.com/sound-speed-liquids-d_715.html

# time discretization
time = 1       # total time in seconds
nt = 400        # number of time steps
dt = (dx**2*dy**2/(dx**2 + dy**2))**(1/2) # time increment

In [3]:
def init_cond(ny,nx,Lx,Ly,X,Y):
    """Creates initial conditions for 2D wave, a Guassian function centered on
        the top edge of the box and spread out over the middle.
    
    Parameters:
    -----------
    ny: int
        number of spatial steps in vertical direction
    nx: int 
        number of spatial steps in horizontal direction
    Lx: int
        total length in x direction
    Ly: int
        total length in y direction
    X:  2D array of float
        meshgrid in x
    Y:  2D array of float
        meshgrin in y
        
    Returns:
    --------
    pi: 2D array of floats
        initial gaussian pressure distribution
        indexing: pi[vertical space step, horizontal space step]
    """
    # Gaussian wave function, f 
    a = 1         # amplitude of wave
    sprx = Lx*0.7   # spread of wave in x dir
    spry = 0.2*Ly # spread of wave in y dir
    x0 = Lx/2     # center of wave
    y0 = Ly       # center of wave
    f = numpy.zeros((ny,nx))
    f = a*numpy.exp(-((X-x0)**2/(2*sprx**2) + (Y-y0)**2/(2*spry**2)))
    #f = numpy.exp(-10*(X**2 + Y**2))
    
    pi = numpy.zeros((ny,nx)) # initiallize pressure array
    pi = f                    # assign f to initial pressure array
    
    return pi

In [4]:
def solve_wave(pi, dy, dx, nt, dt, c):
    """Solves the 2D wave equation using the ______ scheme
    
    Parameters:
    -----------
    
    Returns:
    --------
    
    """
    # these help to simplify expression (s = sigma)
    sx = c*dt/dx
    sy = c*dt/dy
    
    # initialize pressure array
    p = numpy.zeros((nt,ny,nx))
    p[0] = pi.copy()             # copy initial Gaussian wave
    
    """_____________________________begin_first_time_step________________________________"""
    # For first time step:
    g = numpy.ones((ny,nx))*-1     # 2nd initial condition du/dt = g(x,y)
    p[1] = p[0]
    p[1,1:-1,1:-1] = 1/2*sx**2*(p[0,1:-1,2:] + p[0,1:-1,:-2]) +\
                     1/2*sy**2*(p[0,2:,1:-1] + p[0,:-2,1:-1]) -\
                     (1 - sx**2 - sy**2)*p[0,1:-1,1:-1] +\
                     dt*g[1:-1,1:-1]
    
    # Neumann boundary conditions for the first step
    # left boundary, i = 0
    p[1,1:-1,0] = (2*sx**2*p[0,1:-1,1] +\
                   sy**2*(p[0,2:,0] + p[0,:-2,0]) +\
                   2*(1 - sx**2 - sy**2)*p[0,1:-1,0])*0.5 +\
                   dt*g[1:-1,0]

    # right boundary, i = nx          
    p[1,1:-1,-1] = (2*sx**2*p[0,1:-1,-2] +\
                   sy**2*(p[0,2:,-1] + p[0,:-2,-1]) +\
                   2*(1 - sx**2 - sy**2)*p[0,1:-1,-1])*0.5 +\
                   dt*g[1:-1,-1] 

    # bottom boundary, j = 0
    p[1,0,1:-1] = (sx**2*(p[0,0,2:] + p[0,0,:-2]) +\
                  2*sy**2*p[0,1,1:-1] +\
                  2*(1 - sx**2 - sy**2)*p[0,0,1:-1])*0.5 +\
                  dt*g[0,1:-1]

    # top boundary, j = ny
    p[1,-1,1:-1] = (sx**2*(p[0,-1,2:] + p[0,-1,:-2]) +\
                   2*sy**2*p[0,-2,1:-1] +\
                   2*(1 - sx**2 - sy**2)*p[0,-1,1:-1])*0.5 +\
                   dt*g[-1,1:-1]

    # bottom left corner, i = 0, j = 0
    p[1,0,0] = (2*sx**2*p[0,0,1] +\
               2*sy**2*p[0,1,0] +\
               2*(1 - sx**2 - sy**2)*p[0,0,0])*0.5 +\
               dt*g[0,0]

    # top left corner, i = 0, j = ny
    p[1,-1,0] = (2*sx**2*p[0,-1,1] +\
                2*sy**2*p[0,-2,0] +\
                2*(1 - sx**2 - sy**2)*p[0,-1,0])*0.5 +\
                dt*g[-1,0]

    # bottom right corner, i = nx, j = 0
    p[1,0,-1] = (2*sx**2*p[0,0,-1] +\
                2*sy**2*p[0,1,-1] +\
                2*(1 - sx**2 - sy**2)*p[0,0,-1])*0.5 +\
                dt*g[0,-1]

    # top right corner, i = nx, j = ny
    p[1,-1,-1] = (2*sx**2*p[0,-1,-2] +\
                 2*sy**2*p[0,-2,-1] +\
                 2*(1 - sx**2 - sy**2)*p[0,-1,-1])*0.5 +\
                 dt*g[-1,-1]
    
    """_____________________________end_first_time_step________________________________"""
                
    for n in range(1,nt-1):
        
        # iterate through time steps    
        p[n+1,1:-1,1:-1] = sx**2*(p[n,1:-1,2:] + p[n,1:-1,:-2]) +\
                           sy**2*(p[n,2:,1:-1] + p[n,:-2,1:-1]) +\
                           2*(1 - sx**2 - sy**2)*p[n,1:-1,1:-1] - \
                           p[n-1,1:-1,1:-1]
        
        
        # Apply zero Neumann boundary conditions on all edges and corners
        # left boundary, i = 0
        p[n+1,1:-1,0] = 2*sx**2*p[n,1:-1,1] +\
                        sy**2*(p[n,2:,0] + p[n,:-2,0]) +\
                        2*(1 - sx**2 - sy**2)*p[n,1:-1,0] -\
                        p[n-1,1:-1,0]  

        # right boundary, i = nx          
        p[n+1,1:-1,-1] = 2*sx**2*p[n,1:-1,-2] +\
                         sy**2*(p[n,2:,-1] + p[n,:-2,-1]) +\
                         2*(1 - sx**2 - sy**2)*p[n,1:-1,-1] -\
                         p[n-1,1:-1,-1] 
    
        # bottom boundary, j = 0
        p[n+1,0,1:-1] = sx**2*(p[n,0,2:] + p[n,0,:-2]) +\
                        2*sy**2*p[n,1,1:-1] +\
                        2*(1 - sx**2 - sy**2)*p[n,0,1:-1] -\
                        p[n-1,0,1:-1]

        # top boundary, j = ny
        p[n+1,-1,1:-1] = sx**2*(p[n,-1,2:] + p[n,-1,:-2]) +\
                         2*sy**2*p[n,-2,1:-1] +\
                         2*(1 - sx**2 - sy**2)*p[n,-1,1:-1] -\
                         p[n-1,-1,1:-1]
                    
        # bottom left corner, i = 0, j = 0
        p[n+1,0,0] = 2*sx**2*p[n,0,1] +\
                     2*sy**2*p[n,1,0] +\
                     2*(1 - sx**2 - sy**2)*p[n,0,0] -\
                     p[n-1,0,0]
                    
        # top left corner, i = 0, j = ny
        p[n+1,-1,0] = 2*sx**2*p[n,-1,1] +\
                      2*sy**2*p[n,-2,0] +\
                      2*(1 - sx**2 - sy**2)*p[n,-1,0] -\
                      p[n-1,-1,0]
                    
        # bottom right corner, i = nx, j = 0
        p[n+1,0,-1] = 2*sx**2*p[n,0,-1] +\
                      2*sy**2*p[n,1,-1] +\
                      2*(1 - sx**2 - sy**2)*p[n,0,-1] -\
                      p[n-1,0,-1]
                    
        # top right corner, i = nx, j = ny
        p[n+1,-1,-1] = 2*sx**2*p[n,-1,-2] +\
                       2*sy**2*p[n,-2,-1] +\
                       2*(1 - sx**2 - sy**2)*p[n,-1,-1] -\
                       p[n-1,-1,-1]
        
        
        
        
        """If BCs are simple"""
        # Apply zero Neumann boundary conditions
        #p[n+1,:,0] = p[n+1,:,1]   # left boundary
        #p[n+1,:,-1] = p[n+1,:,-2] # right boundary
        #p[n+1,0,:] = p[n+1,1,:]   # bottom boundary
        #p[n+1,-1,:] = p[n+1,-2,:] # top boundary
        
        # apply Dirichlet boundary conditions
        #p[n+1,:,0] = 0  # left wall
        #p[n+1,:,-1] = 0 # right wall
        #p[n+1,0,:] = 0  # bottom wall
        #p[n+1,-1,:] = 0 # top wall
        
    return p

In [21]:
#sigma = 0.1
#dt = (dx**2*dy**2/(dx**2 + dy**2))**(1/2)
pi = init_cond(ny,nx,Lx,Ly,X,Y)
p = solve_wave(pi.copy(),dy,dx,nt,dt,c)

# TESTING

In [6]:
numpy.shape(p[0,1:-1,1:-1])

(49, 149)

In [7]:
c*dt/dx

0.7682212795973759

In [8]:
pi = init_cond(ny,nx,Lx,Ly,X,Y)
fig = pyplot.figure()
ax = fig.add_subplot(111,projection='3d')
ax.plot_surface(X,Y,p[30],rstride=5,cstride=5,cmap=cm.viridis)
ax.set_xlim(0,10)
ax.set_ylim(0,4)
ax.set_xlabel('x');

In [9]:
fig = pyplot.figure(figsize=(8,12))
ax1 = fig.add_subplot(411)
ax2 = fig.add_subplot(412)
ax3 = fig.add_subplot(413)
ax1.contourf(X,Y,p[0],levels=numpy.linspace(0,1,50),cmap=cm.viridis)
ax2.contourf(X,Y,p[1],levels=numpy.linspace(0,1,50),cmap=cm.viridis)
ax3.contourf(X,Y,p[2],levels=numpy.linspace(0,1,50),cmap=cm.viridis);

In [10]:
fig = pyplot.figure()
ax = fig.add_subplot(111,projection='3d')
ax.plot_surface(X,Y,p[1],rstride=5,cstride=5,cmap=cm.viridis)
ax.set_xlabel('x')
ax.set_ylabel('y');
ax.view_init(45,-45)
pyplot.draw()

In [11]:
tt = numpy.zeros(nt)
for i in range(0,nt):
    tt[i] = p[i,-1,:].min()
pyplot.figure()
pyplot.plot(numpy.arange(0,nt),tt);

In [12]:
fig = pyplot.figure(figsize=(8,3))
pyplot.xticks([]), pyplot.yticks([]);
im = pyplot.imshow(p[0], cmap=cm.viridis)

In [13]:
def init():
    im.set_data(numpy.zeros((ny,nx)))
    return im,
def animate(data):
    im.set_array(data)
    return im,

In [14]:
anim = animation.FuncAnimation(fig, animate, frames=p, interval=75)

In [15]:
HTML(anim.to_html5_video())

In [18]:
def data(i,p,lines):
    p = p
    ax.clear()
    line = ax.plot_surface(X,Y,p[i],cmap=cm.viridis)
    return line,

In [24]:
%matplotlib qt
fig = pyplot.figure()
ax = fig.add_subplot(111,projection='3d')

Writer = animation.writers['ffmpeg']
writer = Writer(fps=35, metadata=dict(artist='Me'), bitrate=3800)

line = ax.plot_surface(X,Y,p[0],cmap=cm.viridis)
anim = animation.FuncAnimation(fig,data,fargs=(p,line),interval=30,blit=False)
pyplot.show()